<a href="https://colab.research.google.com/github/Nurkyial/DataVisualization/blob/main/lab6_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Алмазова Нуркыял
М23-534

1. Импортировать данные northwind.db в соответствии с кодом (запрос надо будет доработать под задачи):


In [ ]:
import sqlite3
import pandas as pd
con=sqlite3.Connection('/content/drive/MyDrive/визуализация/northwind.db')
df=pd.read_sql('''with revenues as (SELECT
 shipregion,
 customerid,
 orders.orderid,
 orderdate,
  productid,
 (unitprice*quantity*(1-discount)) as revenue
from orders,"order details"
on orders.orderid="order details".orderid
group by orderdate,shipregion,customerid,orders.orderid, productid)
SELECT orderdate,shipregion, customerid,sum(revenue) as revenuedaily, count(orderid) as ordersdaily
from revenues
group by orderdate;''',con=con)

2. Требуется спроектировать дэшборд для бизнес анализа базы  Northwind в соответствии со стандартным фреймворком (см. лекцию). Здесь и далее текстовые описания, схемы, и прочие доп материалы необходимо добавлять в колаб, реализующий дэшборд. Также далее приведены возможные варианты реализации дешборда, витрин и тп. Их можно и нужно дополнить.

*  Предполагается, что с помощью дэшборда заказчик будет изучать динамику суммарных продаж, типовой заказ (например, через средний чек) в разрезе региона продаж, определять топ3 самых продаваемых категорий и товаров, определять топ 3 стран поставки и клиентов, гранулярность всех витрин – недельная. Определить аудиторию - список заказчиков (ролей)


*  Определить метрики для решения задач из 2.1


3. Создать прототип (сделать эскиз)
* нарисовать графики в порядке, который вы считаете необходимым, с пояснением, почему. Можно в paint. Предполагается использование линейных графиков, графиков солнечной вспышки, столбиковых диаграмм, ящичковых диаграмм, иерархических графиков (treemap).
4. Наполнить данными прототип
* Создайте витрину/датасет, описывающий сумму продаж, агрегированных на недельную гранулярность.
* Создайте витрину/датасет, группирующий сумму заказов внутри топ3 категории товаров по топ 3 самым продаваемым товарам
* Создайте витрину/датасет, группирующий сумму заказов по топ 3 странам поставки и топ 3 клиентам
* Создайте витрину/датасет, группирующий средний чек внутри региона продаж (Западная Европа, США, и т.п.) по неделе продажи за весь период продаж


Определение аудитории:

*   Роли пользователей: Руководители продаж, Аналитики, Менеджеры по работе с клиентами.




Метрики для решения задач:
*   Общая выручка (revenue).
*   Средний чек (average order value).
*   Количество заказов (order count).
*   Топовые категории и товары (top categories and products).
*   Топовые страны поставки и клиенты (top shipping countries and customers).

In [ ]:

# Витрина: Сумма продаж, агрегированная на недельную гранулярность
weekly_sales_query = '''
WITH weekly_data AS (
    SELECT
        strftime('%Y-%W', orders.orderdate) AS week,
        SUM("order details".unitprice * "order details".quantity * (1 - "order details".discount)) AS revenue
    FROM orders
    JOIN "order details" ON orders.orderid = "order details".orderid
    GROUP BY strftime('%Y-%W', orders.orderdate)
)
SELECT week, SUM(revenue) AS total_revenue
FROM weekly_data
GROUP BY week
ORDER BY week;
'''
weekly_sales = pd.read_sql(weekly_sales_query, con)

# Витрина: Сумма заказов внутри топ-3 категории товаров по топ-3 самым продаваемым товарам
top_categories_products_query = '''
WITH top_products AS (
    SELECT
        products.categoryid,
        "order details".productid,
        SUM("order details".unitprice * "order details".quantity * (1 - "order details".discount)) AS total_revenue
    FROM "order details"
    JOIN products ON "order details".productid = products.productid
    GROUP BY products.categoryid, "order details".productid
    ORDER BY total_revenue DESC
    LIMIT 3
)
SELECT
    categories.categoryname,
    products.productname,
    SUM("order details".unitprice * "order details".quantity * (1 - "order details".discount)) AS total_revenue
FROM "order details"
JOIN products ON "order details".productid = products.productid
JOIN categories ON products.categoryid = categories.categoryid
WHERE "order details".productid IN (SELECT productid FROM top_products)
GROUP BY categories.categoryname, products.productname
ORDER BY total_revenue DESC;
'''
top_categories_products = pd.read_sql(top_categories_products_query, con)

# Витрина: Сумма заказов по топ-3 странам поставки и топ-3 клиентам
top_countries_customers_query = '''
WITH top_countries AS (
    SELECT
        orders.shipcountry,
        SUM("order details".unitprice * "order details".quantity * (1 - "order details".discount)) AS total_revenue
    FROM orders
    JOIN "order details" ON orders.orderid = "order details".orderid
    GROUP BY orders.shipcountry
    ORDER BY total_revenue DESC
    LIMIT 3
),
top_customers AS (
    SELECT
        orders.customerid,
        SUM("order details".unitprice * "order details".quantity * (1 - "order details".discount)) AS total_revenue
    FROM orders
    JOIN "order details" ON orders.orderid = "order details".orderid
    GROUP BY orders.customerid
    ORDER BY total_revenue DESC
    LIMIT 3
)
SELECT
    orders.shipcountry,
    orders.customerid,
    SUM("order details".unitprice * "order details".quantity * (1 - "order details".discount)) AS total_revenue
FROM orders
JOIN "order details" ON orders.orderid = "order details".orderid
WHERE orders.shipcountry IN (SELECT shipcountry FROM top_countries)
AND orders.customerid IN (SELECT customerid FROM top_customers)
GROUP BY orders.shipcountry, orders.customerid
ORDER BY total_revenue DESC;
'''
top_countries_customers = pd.read_sql(top_countries_customers_query, con)

# Витрина: Средний чек по регионам (недельная гранулярность)
average_check_query = '''
WITH weekly_data AS (
    SELECT
        strftime('%Y-%W', orders.orderdate) AS week,
        orders.shipregion,
        SUM("order details".unitprice * "order details".quantity * (1 - "order details".discount)) AS revenue,
        COUNT(DISTINCT orders.orderid) AS order_count
    FROM orders
    JOIN "order details" ON orders.orderid = "order details".orderid
    GROUP BY strftime('%Y-%W', orders.orderdate), orders.shipregion
)
SELECT
    week,
    shipregion,
    SUM(revenue) / SUM(order_count) AS avg_check
FROM weekly_data
GROUP BY week, shipregion
ORDER BY week, shipregion;
'''
average_check = pd.read_sql(average_check_query, con)


In [ ]:
!pip install jupyter-dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.5 MB/s eta 0:00:00


5. Развернуть в прод (создайте дэшборд, используя библиотеку Jupyter Dash) (далее идут примеры использования графиков, можете свои варианты)

In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Dash
import plotly.express as px

app = Dash(__name__)


# 4.1
line_fig = px.line(weekly_sales, x='week', y='total_revenue', title='Динамика суммарных продаж (недельная)')

# 4.2
sunburst_fig = px.sunburst(top_categories_products, path=['CategoryName', 'ProductName'], values='total_revenue', title='Топ-3 самых продаваемых категории и товаров')

# 4.3
treemap_fig = px.treemap(top_countries_customers, path=['ShipCountry', 'CustomerID'], values='total_revenue', title='Топ-3 страны поставки и клиенты')

# 4.4
box_fig = px.box(average_check, x='shipregion', y='avg_check', title='Распределение заказов по регионам')


app.layout = html.Div(children=[
    html.H1(children='Дэшборд Northwind'),
    html.Div(children='Анализ данных Northwind.'),

    dcc.Graph(
        id='line',
        figure=line_fig
    ),

    dcc.Graph(
        id='sunburst',
        figure=sunburst_fig
    ),

    dcc.Graph(
        id='treemap',
        figure=treemap_fig
    ),

    dcc.Graph(
        id='box',
        figure=box_fig
    )

])

if __name__ == '__main__':
    app.run(mode='inline')

<IPython.core.display.Javascript object>

6. Здесь и далее, в порядке учета обратной связи от заказчика - добавьте интерактивные элементы:

6.1 Слайдер выбора года, для графика из 5.1

6.2Выпадающий список страны поставки для 5.3


In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Dash, Input, Output
import plotly.express as px

app = Dash(__name__)


# 4.1
line_fig = px.line(weekly_sales, x='week', y='total_revenue', title='Динамика суммарных продаж (недельная)')

# 4.2
sunburst_fig = px.sunburst(top_categories_products, path=['CategoryName', 'ProductName'], values='total_revenue', title='Топ-3 самых продаваемых категории и товаров')

# 4.3
treemap_fig = px.treemap(top_countries_customers, path=['ShipCountry', 'CustomerID'], values='total_revenue', title='Топ-3 страны поставки и клиенты')

# 4.4
box_fig = px.box(average_check, x='shipregion', y='avg_check', title='Распределение заказов по регионам')


app.layout = html.Div(children=[
    html.H1(children='Дэшборд Northwind'),
    html.Div(children='Анализ данных Northwind.'),

    dcc.Slider(
        id='year-slider',
        min=weekly_sales['week'].apply(lambda x: int(x[:4])).min(),
        max=weekly_sales['week'].apply(lambda x: int(x[:4])).max(),
        value=weekly_sales['week'].apply(lambda x: int(x[:4])).min(),
        marks={str(year): str(year) for year in weekly_sales['week'].apply(lambda x: int(x[:4])).unique()},
        step=None
    ),
    dcc.Graph(id='line-graph'),

    dcc.Graph(
        id='sunburst',
        figure=sunburst_fig
    ),

    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in top_countries_customers['ShipCountry'].unique()],
        value=top_countries_customers['ShipCountry'].unique()[0]
    ),
    dcc.Graph(id='bar-graph'),


    dcc.Graph(
        id='box',
        figure=box_fig
    )
])

@app.callback(
        Output('line-graph', 'figure'),
        Input('year-slider', 'value')
    )
def update_line_graph(selected_year):
    filtered_df = weekly_sales[weekly_sales['week'].apply(lambda x: int(x[:4])) == selected_year]
    fig = px.line(filtered_df, x='week', y='total_revenue', title=f'Динамика суммарных продаж (недельная) для {selected_year}')
    return fig

@app.callback(
    Output('bar-graph', 'figure'),
    Input('country-dropdown', 'value')
    )
def update_bar_graph(selected_country):
    filtered_df = top_countries_customers[top_countries_customers['ShipCountry'] == selected_country]
    fig = px.bar(filtered_df, x='CustomerID', y='total_revenue', title=f'Топ клиенты для страны {selected_country}')
    return fig



if __name__ == '__main__':
    app.run(mode='inline')

<IPython.core.display.Javascript object>

In [ ]:


weekly_sales['anomaly'] = (weekly_sales['total_revenue'] > weekly_sales['total_revenue'].mean() + 2 * weekly_sales['total_revenue'].std()) | (weekly_sales['total_revenue'] < weekly_sales['total_revenue'].mean() - 2 * weekly_sales['total_revenue'].std())

app = JupyterDash(__name__)


line_fig = px.line(weekly_sales, x='week', y='total_revenue', title='Динамика суммарных продаж (недельная)',
                   labels={'week': 'Неделя', 'total_revenue': 'Суммарные продажи'})
line_fig.update_layout(font=dict(family='Arial'), plot_bgcolor='rgba(230, 230, 230, 0.5)')
line_fig.add_scatter(x=weekly_sales[weekly_sales['anomaly']]['week'], y=weekly_sales[weekly_sales['anomaly']]['total_revenue'],
                     mode='markers', marker=dict(color='red', size=10), name='Аномалия')

sunburst_fig = px.sunburst(top_categories_products, path=['CategoryName', 'ProductName'], values='total_revenue', title='Топ-3 самых продаваемых категории и товаров')
sunburst_fig.update_layout(font=dict(family='Arial'), plot_bgcolor='rgba(230, 230, 230, 0.5)')

treemap_fig = px.treemap(top_countries_customers, path=['ShipCountry', 'CustomerID'], values='total_revenue', title='Топ-3 страны поставки и клиенты')
treemap_fig.update_layout(font=dict(family='Arial'), plot_bgcolor='rgba(230, 230, 230, 0.5)')

box_fig = px.box(average_check, x='shipregion', y='avg_check', title='Распределение заказов по регионам', labels={'shipregion': 'Регион', 'avg_check': 'Средний чек'})
box_fig.update_layout(font=dict(family='Arial'), plot_bgcolor='rgba(230, 230, 230, 0.5)')


app.layout = html.Div(children=[
    html.H1(children='Дэшборд Northwind'),
    html.Div(children='Анализ данных Northwind.'),

    # Слайдер выбора года для 5.1
    dcc.Slider(
        id='year-slider',
        min=weekly_sales['week'].apply(lambda x: int(x[:4])).min(),
        max=weekly_sales['week'].apply(lambda x: int(x[:4])).max(),
        value=weekly_sales['week'].apply(lambda x: int(x[:4])).min(),
        marks={str(year): str(year) for year in weekly_sales['week'].apply(lambda x: int(x[:4])).unique()},
        step=None
    ),
    dcc.Graph(id='line-graph'),

    # Диаграмма солнечной вспышки для 5.2
    dcc.Graph(
        id='sunburst',
        figure=sunburst_fig
    ),

    # Выпадающий список страны поставки для 5.3
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in top_countries_customers['ShipCountry'].unique()],
        value=top_countries_customers['ShipCountry'].unique()[0]
    ),
    dcc.Graph(id='bar-graph'),

    # Ящичковая диаграмма для 5.4
    dcc.Graph(
        id='box',
        figure=box_fig
    )
])


@app.callback(
    Output('line-graph', 'figure'),
    Input('year-slider', 'value')
)
def update_line_graph(selected_year):
    filtered_df = weekly_sales[weekly_sales['week'].apply(lambda x: int(x[:4])) == selected_year]
    fig = px.line(filtered_df, x='week', y='total_revenue', title=f'Динамика суммарных продаж (недельная) для {selected_year}',
                  labels={'week': 'Неделя', 'total_revenue': 'Суммарные продажи'})
    fig.update_layout(font=dict(family='Arial'), plot_bgcolor='rgba(230, 230, 230, 0.5)')
    fig.add_scatter(x=filtered_df[filtered_df['anomaly']]['week'], y=filtered_df[filtered_df['anomaly']]['total_revenue'],
                    mode='markers', marker=dict(color='red', size=10), name='Аномалия')
    return fig

@app.callback(
    Output('bar-graph', 'figure'),
    Input('country-dropdown', 'value')
)
def update_bar_graph(selected_country):
    filtered_df = top_countries_customers[top_countries_customers['shipcountry'] == selected_country]
    fig = px.bar(filtered_df, x='customerid', y='total_revenue', title=f'Топ клиенты для страны {selected_country}',
                 labels={'customerid': 'Клиент', 'total_revenue': 'Суммарные продажи'})
    fig.update_layout(font=dict(family='Arial'), plot_bgcolor='rgba(230, 230, 230, 0.5)')
    return fig

if __name__ == '__main__':
    app.run(mode='inline')


/usr/local/lib/python3.10/dist-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>